# Germini AI Demo

## Setup

In [1]:
# As dotenv does not load so reliably, we do it ourselves.
from os import environ

def load_dotenv():
    """Load all the valies in the.env file into the environment."""
    with open(".env", "r") as f:
        for line in f.readlines():
            parts = line.strip().split("=")
            environ[parts[0]] = parts[1]

load_dotenv()


## Basic Prompting

This notebook contains some play code and interaction with Gemini AI via the 
Gemini API.

In [2]:
from google import genai
import os

GOOGLE_API_KEY=os.environ['GOOGLE_API_KEY']

client = genai.Client(api_key=GOOGLE_API_KEY)
def get_answer(prompt: str) -> str:
    return client.models.generate_content(
        model="gemini-2.0-flash", contents=prompt)


In [3]:
# By deafult, the client calls not not track history
response1 = get_answer("How many simple colours are there?")
print("response: " + str(response1))
response1 = get_answer("And of those colours, how many start with the letter 'b'?")
print("response: " + str(response1))


response: candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='The question of how many "simple" colors there are is a bit tricky because it depends on what you mean by "simple." Here\'s a breakdown of different ways to think about it:\n\n**1. Primary Colors (Traditional Subtractive):**\n\n*   **Red, Yellow, Blue:** These are the colors traditionally considered the primary colors in art and pigment mixing. All other colors can be created by mixing these. So, by this definition, there are **3** simple colors.\n\n**2. Primary Colors (Additive):**\n\n*   **Red, Green, Blue:** These are the primary colors of light (used in screens like TVs and computer monitors). Mixing these in different proportions creates a wide range of colors. So, by this definition, there are **3** simple colors.\n\n**3. Psychological Primaries (Unique H

In [ ]:
# Creating a chat
from google import genai

chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message("How many simple colours are there?")
response = chat.send_message(
    "And of those colours, how many start with the letter 'b'?")
print(response.text)
# for message in chat._curated_history:
#     print(f'role - {message.role}' end=": ")
#     print(message.parts[0].text)

## Configure text generation

In [ ]:
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["What is 1 + 1?"],
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1
    )
)
print(response.text)

## Adding Instructions

In [ ]:
## Add system instructions
# System instructions let you steer the behavior of a model based on your s
# pecific needs and use cases. You provide the model additional context to
# understand the task, generate more customized responses, and adhere to
# specific guidelines over the full user interaction with the model.

sys_instruct="You are a cat. Your name is Neko."
# client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=sys_instruct),
    contents=["What is 1 + 1?"]
)
print(response.text)

## Audio Processing

In [15]:
from google import genai

client = genai.Client()

myfile = client.files.upload(file='media/Nataly.ogg')

response = client.models.generate_content(
  model='gemini-2.0-flash',
  contents=['Describe this audio clip', myfile]
)

print(response.text)

The audio clip appears to be a brief voicemail message left by a female speaker. She is inquiring if the recipient of the message is at home and if she and possibly someone else could come by that afternoon. She ends the message by saying "Thanks, Bye."


## Code Execution

In [13]:
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
  model='gemini-2.0-flash',
  contents='What is the sum of the first 50 prime numbers? '
           'Generate and run code for the calculation, and make sure you get all 50.',
  config=types.GenerateContentConfig(
    tools=[types.Tool(
      code_execution=types.ToolCodeExecution
    )]
  )
)

print("Text: " + response.text)
print("CodeExec: " + str(response.code_execution_result))
print("Code: " + str(response.executable_code))

# Use code execution in chat
# You can also use code execution as part of a chat.

chat = client.chats.create(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
      tools=[types.Tool(
        code_execution=types.ToolCodeExecution
      )]
    )
)

response = chat.send_message(
    "Can you run some code to bogo-sort this list of numbers?: [2,34,1,65,4]")
print("1. " + response.text)

response = chat.send_message("Yes, please create the code.")
print("2. " + response.text)


Text: Okay, I need to find the first 50 prime numbers and then calculate their sum. I'll use a Python code block to accomplish this. I'll start by defining a function to check if a number is prime, and then I'll generate a list of the first 50 prime numbers. Finally, I'll sum them up.

The sum of the first 50 prime numbers is 5117.

CodeExec: primes=[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229]
sum_of_primes=5117

Code: def is_prime(n):
  """Check if a number is prime."""
  if n <= 1:
    return False
  for i in range(2, int(n**0.5) + 1):
    if n % i == 0:
      return False
  return True

primes = []
num = 2
while len(primes) < 50:
  if is_prime(num):
    primes.append(num)
  num += 1

sum_of_primes = sum(primes)
print(f'{primes=}')
print(f'{sum_of_primes=}')

1. I can't actually *run* code to bogo-sort a list 